In [2]:
import numpy as np
from Levenshtein import distance
from tqdm import tqdm
import os

In [7]:
paths = os.listdir('./data')
paths = [path for path in paths if path.endswith('.txt')]
paths_data = ['./data/' + path for path in paths]
paths

['common_words.txt',
 'english.txt',
 'hungarian_text_0.txt',
 'hungarian_text_unduped_0.txt']

In [2]:
# load ./data/hungarian_data_0.txt
with open('./data/hungarian_text_0.txt', encoding="utf-8") as f:
    data = f.readlines()

In [3]:
data[0:4]

['2DIN fejegység - Pilisszentlászló - 2DIN - 2DIN fejegység, 2DIN navigáció, 2DIN kamera\n',
 'Keresőoptimalizált bérelhető weboldal készítés kontra saját weboldal\n',
 'Az elmúlt időszakban megnövekedett az igény a céges bemutatkozó weboldalakra, valamint a webáruházakra, hiszen egyre kevesebb az a szolgáltató vagy termékeket értékesítő, aki online jelenlét hiányában is képes lenne hosszútávon fennmaradni. A cégek két irányban tudnak gondolkodni: saját vagy bérelhető weboldalban. Sok éve foglalkozom keresőoptimalizált bérelhető weboldal készítéssel, valamint természetesen olyan honlapokkal is, amelyek átadás után véglegesen a vevő tulajdonában maradnak. Az évek során több 100 bemutatkozó honlapot és webáruházat adtam át sikeresen, így jól látom, mikor, melyik lehetőséget érdemesebb inkább választani.\n',
 'Az alábbi sorokkal bízom benne, hogy segíthetek döntést hozni abban, hogy saját weboldalt nyiss vagy inkább keresőoptimalizált bérelhető weboldal készítés szolgáltatást válassz.\n']

In [4]:
def remove_near_duplicates(strings):
    # Create an empty list to store unique strings
    unique_strings = []
    duplicates = []
    
    # Loop through each string in the input list
    for string in strings:
        # Check if the string is similar to any existing unique strings
        is_duplicate = False
        if string == '\n':
            unique_strings.append(string)
            continue
        for unique_string in unique_strings:
            # Calculate the Levenshtein distance between the strings
            lev_distance = distance(string, unique_string)
            
            # Check if the distance is below the given threshold
            threshold = min(len(unique_string), len(string))//2
            if lev_distance <= threshold:
                # print("--------------------------------------------")
                # print(unique_string)
                # print(string)
                is_duplicate = True
                break
        
        # If the string is not a near duplicate, add it to the list of unique strings
        if not is_duplicate:
            unique_strings.append(string)
        else:
            duplicates.append(string)
    
    # Return the list of unique strings
    return unique_strings, duplicates


In [5]:
from concurrent.futures import ThreadPoolExecutor

def remove_near_duplicates_wrapper(data):
    return remove_near_duplicates(data)

WINDOW = 10000
num_processes=16
executor = ThreadPoolExecutor(max_workers=num_processes)
futures = []

for i in range(0, len(data), WINDOW):
    subset = data[i:i+WINDOW].copy()
    future = executor.submit(remove_near_duplicates_wrapper, subset)
    futures.append(future)

unduped = []
for future in futures:
    unique, dupes = future.result()
    unduped.extend(unique)


In [5]:
from multiprocessing import Pool

def remove_near_duplicates_wrapper(data):
    return remove_near_duplicates(data)

WINDOW = 10000
pool = Pool()
results = []

for i in range(0, len(data), WINDOW):
    subset = data[i:i+WINDOW]
    result = pool.apply_async(remove_near_duplicates_wrapper, args=(subset,))
    results.append(result)
print("setup done")
unduped = []
for result in results:
    unique, dupes = result.get()
    print("done")
    unduped.extend(unique)


setup done


In [82]:
unduped = []
WINDOW = 10000
for i in range(0, len(data),WINDOW ):
    unique, dupes = remove_near_duplicates(data[i:i+WINDOW])
    unduped.extend(unique)

 58%|█████▊    | 5844/10000 [02:49<02:00, 34.48it/s] 


KeyboardInterrupt: 

In [76]:
# load ./data/hungarian_data_0.txt
with open('./data/hungarian_text_unduped_0.txt',"w", encoding="utf-8" ) as f:
    f.write(''.join(unduped))
    

In [ ]:
# load ./data/hungarian_data_0.txt
with open('./data/hungarian_text_dupes_0.txt', encoding="utf-8") as f:
    f.write(dupes)

In [65]:
len(dupes)

7068

In [66]:
unique

['2DIN fejegység - Pilisszentlászló - 2DIN - 2DIN fejegység, 2DIN navigáció, 2DIN kamera\n',
 'Keresőoptimalizált bérelhető weboldal készítés kontra saját weboldal\n',
 'Az elmúlt időszakban megnövekedett az igény a céges bemutatkozó weboldalakra, valamint a webáruházakra, hiszen egyre kevesebb az a szolgáltató vagy termékeket értékesítő, aki online jelenlét hiányában is képes lenne hosszútávon fennmaradni. A cégek két irányban tudnak gondolkodni: saját vagy bérelhető weboldalban. Sok éve foglalkozom keresőoptimalizált bérelhető weboldal készítéssel, valamint természetesen olyan honlapokkal is, amelyek átadás után véglegesen a vevő tulajdonában maradnak. Az évek során több 100 bemutatkozó honlapot és webáruházat adtam át sikeresen, így jól látom, mikor, melyik lehetőséget érdemesebb inkább választani.\n',
 'Az alábbi sorokkal bízom benne, hogy segíthetek döntést hozni abban, hogy saját weboldalt nyiss vagy inkább keresőoptimalizált bérelhető weboldal készítés szolgáltatást válassz.\n',

In [77]:
dupes

['1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 november 2022 december 2022 január 2023 február 2023 március 2023 április 2023 május 2023 június 2023 július 2023 augusztus 2023 szeptember 2023 október 2023\n',
 '${$T.render.getIconImage($T.mgr,($T.item.inbound.dep?$T.item.inbound.dep:$T.item.dest),false)} ${$T.mgr.getAptName(($T.item.inbound.dep?$T.item.inbound.dep:$T.item.dest))} ${$T.render.getDateTime($T.item.inbound,false)} ${$T.mgr.getAptName(($T.item.inbound.dest?$T.item.inbound.dest:$T.item.dep))} ${$T.render.getDateTime($T.item.inbound,true)} ${$T.render.getDurationAndStops($T.item.inbound)} ${$T.mgr.getAirline($T.item.inbound, $T.item.airline)}\n',
 'Weboldalunk az alapvető működéshez szükséges cookie-kat használ. Szélesebb körű funkcionalitáshoz marketing jellegű cookie-kat engedélyezhet, amivel elfogadja az Adatkezelési tájékoztatóban foglaltakat.\n',
 'Kockázati figyelmeztetés: A CFD-k összetett instrumentumok és a tőkeáttételnek köszön